In [1]:
import os
import sys
import time
import h5py
import numpy as np
from datetime import datetime, timedelta

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
sys.path.insert(0, '/glade/u/home/ksha/GAN_proj/')
sys.path.insert(0, '/glade/u/home/ksha/GAN_proj/libs/')

from namelist import *
import data_utils as du

In [4]:
with h5py.File(save_dir+'CNN_domain.hdf', 'r') as h5io:
    lon_01 = h5io['lon_01'][...]
    lat_01 = h5io['lat_01'][...]

In [5]:
year = 2021
BATCH_dir = '/glade/campaign/cisl/aiml/ksha/BATCH_MRMS/'

In [6]:
with h5py.File('/glade/campaign/cisl/aiml/ksha/GFS/MRMS_y{}.hdf'.format(year), 'r') as h5io:
    MRMS = h5io['MRMS'][...]

In [7]:
size = 128
gap = 24
grid_shape = (256, 576)

In [8]:
N_hour = 3
L = len(MRMS) - N_hour
grid_shape = MRMS.shape[1:]

In [9]:
batch_name = 'MRMS_y{}_day{}_ix{}_iy{}.npy'

for i in range(L):
    mrms = np.zeros(grid_shape)
    for j in range(N_hour):
        mrms += MRMS[i+j]
        
    if np.sum(np.isnan(mrms)) == 0:
        for ix in range(0, grid_shape[0]+gap, gap):
            for iy in range(0, grid_shape[1]+gap, gap):
                ix_start = ix
                ix_end = ix+size
    
                iy_start = iy
                iy_end = iy+size
    
                if (ix_end < grid_shape[0]) and (iy_end < grid_shape[1]):
                    mrms_save = mrms[ix_start:ix_end, iy_start:iy_end]
    
                    save_name = BATCH_dir+batch_name.format(year, i, ix, iy)
                    #print(save_name)
                    np.save(save_name, mrms_save)